In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Obtaining dataframe from csv format which was downloaded from Kaggle
df = pd.read_csv('Combined_News_DJIA.csv',encoding = "ISO-8859-1")

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
#checking null values in dataset
df.info()

In [ ]:
df.isna().sum()

In [ ]:
train = df[df['Date']<'2015-01-01']
test = df[df['Date']>'2014-12-31']

In [ ]:
# Removing punctuations
data=train.iloc[:,2:27]
data.replace("[^a-zA-Z]"," ",regex=True, inplace=True)

# Renaming column names for ease of access
list1= [i for i in range(25)]
new_Index=[str(i) for i in list1]
data.columns= new_Index
data.head(5)
    


In [ ]:
# Convertng headlines to lower case
for index in new_Index:
    data[index]=data[index].str.lower()
data.head(1)

In [ ]:
' '.join(str(x) for x in data.iloc[1,0:25])


In [ ]:
headlines = []
for row in range(0,len(data.index)):
    headlines.append(' '.join(str(x) for x in data.iloc[row,0:25]))

In [ ]:
headlines[0]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier


In [ ]:
## implement BAG OF WORDS
countvector=CountVectorizer(ngram_range=(2,2))
traindataset=countvector.fit_transform(headlines)

In [ ]:
# implement RandomForest Classifier
randomclassifier=RandomForestClassifier(n_estimators=200,criterion='entropy')
randomclassifier.fit(traindataset,train['Label'])

In [ ]:
## Predict for the Test Dataset
test_transform= []
for row in range(0,len(test.index)):
    test_transform.append(' '.join(str(x) for x in test.iloc[row,2:27]))
test_dataset = countvector.transform(test_transform)
predictions = randomclassifier.predict(test_dataset)

In [ ]:
## Import library to check accuracy
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [ ]:
matrix=confusion_matrix(test['Label'],predictions)
print(matrix)
score=accuracy_score(test['Label'],predictions)
print(score)
report=classification_report(test['Label'],predictions)
print(report)

In [ ]:
from flask import Flask,request,render_template
import pickle

In [ ]:
app = Flask('__name__')

In [ ]:
q = ""

In [ ]:
#app.route('/')
def loadPage():
    return render_template('home.html',query="")
    

In [ ]:
#app.route('/predict',method=['post'])
def predict():
    inputQuery1 = request.form['query1']
    inputQuery2 = request.form['query2']
    inputQuery3 = request.form['query3']
    inputQuery4 = request.form['query4']
    inputQuery5 = request.form['query5']
    
    model = pickle.load(open('model.sav','rb'))
    
    data = [[inputQuery1,inputQuery2,inputQuery3,inputQuery4,inputQuery5]]
    new_df = pd.DataFrame(data,columns=['texture_mean','perimeter_mean','smoothness_mean','compactness_mean','symmetry_mean'])
    
    single = model.predict(new_df)
    probability = model.predict_proba(new_df)[:,1]
    
    if single==1:
        o1 = 'The stockprice increased'
        o2 = 'Confidence: {}'.format(probablity*100)
    else:
        o1 = 'The stockprice remains same or decreased'
        o2 = 'Confidence: {}'.format(probablity*100)
    
    return render_template('home.html',output1=o1, output2=o2, query1 = request.form['query1'], query2 = request.form['query2'], query3 = request.form['query3'], query4 = request.form['query4'], query5 = request.form['query5'])


In [ ]:
if __name__ == '__main__':
    app.run()